In [32]:
from abc import ABC
import torch
import torch.nn as nn

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [34]:
x0, y0 = torch.normal(mean=1.7, std=1, size=(100, 2)) + 1, torch.zeros(100)  # 数据集1
x1, y1 = torch.normal(mean=-1.7, std=1, size=(100, 2)) + 1, torch.ones(100)  # 数据集2
train_x, train_y = torch.cat((x0, x1), 0), torch.cat((y0, y1), 0)
train_x, train_y = train_x.to(device), train_y.to(device)


class LR(nn.Module, ABC):
    def __init__(self):
        super(LR, self).__init__()
        self.features = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.features(x)
        x = self.sigmoid(x)
        return x


lr_net = LR()
lr_net = lr_net.to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(lr_net.parameters(), lr=0.01, momentum=0.9)

for iteration in range(500):
    optimizer.zero_grad()  # 梯度清零
    y_pred = lr_net(train_x)
    loss = loss_fn(y_pred.squeeze(), train_y)
    loss.backward()
    optimizer.step()

    if iteration % 20 == 0:
        print("iteration：{iteration},    loss: {loss}".format(iteration=iteration, loss=loss))

iteration：0,    loss: 0.7025330662727356
iteration：20,    loss: 0.18199969828128815
iteration：40,    loss: 0.13756251335144043
iteration：60,    loss: 0.12247955054044724
iteration：80,    loss: 0.11182247847318649
iteration：100,    loss: 0.10344232618808746
iteration：120,    loss: 0.09663589298725128
iteration：140,    loss: 0.0909983217716217
iteration：160,    loss: 0.08625487238168716
iteration：180,    loss: 0.0822082906961441
iteration：200,    loss: 0.07871318608522415
iteration：220,    loss: 0.0756606012582779
iteration：240,    loss: 0.07296784967184067
iteration：260,    loss: 0.07057137042284012
iteration：280,    loss: 0.06842180341482162
iteration：300,    loss: 0.06648018956184387
iteration：320,    loss: 0.06471557170152664
iteration：340,    loss: 0.06310289353132248
iteration：360,    loss: 0.06162182614207268
iteration：380,    loss: 0.06025553494691849
iteration：400,    loss: 0.05899008736014366
iteration：420,    loss: 0.05781375616788864
iteration：440,    loss: 0.0567166134715080

In [35]:
torch.save(lr_net, 'torch_model.bin')  # 保存整个模型(也保存了模型所在设备情况)

In [36]:
# 注意:
# 方式一:导入模型类定义
# 方式二:模型类定义与模型加载位于同一文件内
lr_net_new = torch.load('torch_model.bin')  # 方式二
lr_net_new

LR(
  (features): Linear(in_features=2, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [37]:
next(lr_net_new.parameters()).device


device(type='cuda', index=0)